** Assumptions ** 
1. RH. Deriving RH from specific humidity (approximating the mixing ratio w as q) 
2. Climate class. The NFDRS method uses climate class to approximate a precipitation rate, which is 0.25 in/hr for climate classes 1 and 2 and 0.05 in/hr for climate classes 3 and 4. The amount of variability in daily DFM is very dependent on this. The precip rate is then used to determine the precip duration, which cannot exceed 8 hours if this method is used. 
3. SATVPD. Using saturation vapor pressure evaluated at the average daily temperature rather than observed temperature (dry-bulb temperature of afternoon observation time). 
4. AMBVP. Using average RH (derived from q) rather than observed RH. 
5. EMC. Using Tmin for dew point temperature and RH (derived from q). 

** METHOD **

The method used here is outlined in the National Fire Danger Rating system: basic equations article. It can be found here: http://www.fs.fed.us/psw/publications/documents/psw_gtr082/psw_gtr082.pdf
Citation: Cohen, JD and Deeming JE. 1985. The National Fire Danger Rating System: basic equations. Pacific Southwest Forest and Range Experiment Station, Berkeley, CA. 

Essentially the approach draws on regression equations developed by Simard (1968) on the basis of tables in the Wood Handbook (US Forest Products Laboratory 1955, revised 1974). EMC is expressed as a percent moisture content. 

The citation for Simard (1968) is: Simard AJ (1968). The moisture content of forest fuels I: A review of basic concepts. Ottawa, ONT: Forest Fire Research Institute, 1968. Rep. FF-X-14, 47p. 




** EQUATIONS ** 

$$ e_s(T) = 611 * \exp( \frac {17.67 * (T - T_0)} { T - 29.65} ) $$ where $T_0$ = 273.15 K 
$$ p(h) = 101325 * (1 - (2.25577 * 10^-5) * h)^{5.25588} $$ where h = elevation (in meters)
$$ AMBVP = \frac {RH * e_{s_{min}}} {100} $$ where AMBVP = ambient vapor pressure  
$$ SATVPD = e_s $$ where SATVPD = saturation vapor pressure for $T_{dew}$ 
$$ SATVPN = e_s (T_{min}) $$ where SATVPN = saturation vapor pressure for $T_{min}$
$$ SATVPX = e_s (T_{max}) $$ where SATVPx = saturation vapor pressure for $T_{max}$
$$ RH = 100.0 * \frac {w} {w_s} $$ where $$ w_s = 0.622 * \frac {e_s} {p} $$ and w = q (approximating the mixing ratio as the specific humidity) 
$$ RH_{max} = 100.0 * \frac {AMBVP} {SATVPN} $$
$$ RH_{min} = 100.0 * \frac {AMBVP} {SATVPX} $$

$$ PPTDUR(PPTAMT,WETRAT) = \frac {PPTAMT} {WETRAT} + 0.49 $$ where PPTDUR = duration of precipitation (in hours), WETRAT = precipitation rate (in in/hr) and PPTAMT = amount of precipitation (in inches). PPTDUR is rounded to the nearest integer. 

if RH < 10, then 
$$ EMC(RH,T) = 0.03229 + (0.281073 * RH) - (0.000578 * T * RH) $$

if RH $\ge$ 10 and RH $\le$ 50, then 

$$ EMC(RH,T) = 2.22749 + (0.160107*RH) - (0.014784*T) $$ 

if RH > 50, then

$$ EMC(RH,T) = 21.0606 + (0.005565*(RH^2)) - (0.00035*RH*T) - (0.483199*RH) $$ where T is in Fahrenheit 

$$ EMC_{min} = EMC(RH_{min},T_{max}) $$ 
$$ EMC = EMC(RH,T_{min}) $$ 
$$ EMC_{max} = EMC(RH_{max},T_{min}) $$

$$ PHI = lat * 0.01745 $$ where lat is in degrees
$$ DECL = 0.41008 * \sin((julian - 82) * 0.01745) $$ where julian is the Julian day of the year
$$ DAYLIT = 24 * (1.0 - \frac{\arccos(\tan(PHI)*\tan(DECL))}{3.1416}) $$ where DAYLIT = number of hours between sunrise and sunset
$$ EMC_{bar} = \frac {(DAYLIT * EMC_{min}) + (24 - DAYLIT) * EMC_{max} )} {24.0} $$ where $EMC_{bar}$ is the weighted 24-hour average EMC
$$ BNDRYH = \frac {(24.0 - PPTDUR) * EMC_{bar} + PPTDUR * (0.5 * PPTDUR + 41.0)} {24.0} $$ where BNDRYH = weighted 24-hour average boundary condition 
$$ MC100 = YMC100 + (BNDRYH - YMC100) * (1.0 - 0.87 * \exp(-0.24)) $$ where $ YMC100 = 5.0 + (5.0 * CLIMAT) $ to initialize a computational period and is otherwise the MC100 value calculated the previous day (and CLIMAT is the climate class, e.g. 1,2,3, or 4) 


** MY APPROACH ** 

To understand how the algorithm for 100-hr DFM was performing, I imported DFM data available through UI Metdata to compare with my algorithm for DFM using Integrated Scenarios inputs. I compared for one gridcell located in the WA Cascades, with slightly different resolutions between the UI Metdata DFM and Integrated Scenarios. I coded this up in Python (Ipython notebook available for reference). 

Since there was a rather low correlation between UI Metdata DFM and my 100-hr DFM algorithm (using the NFDRS approach), I attempted to isolate the differences by downloading UI Metdata inputs and using those instead of the IS data inputs. 

I should mention that this was done for one water year, 1982, and one gridcell, as previously mentioned.

Initially, I used the same UI Metdata inputs as I was using from IS, Tmax, Tmin, q (specific humidity), and Precip. To further isolate potential issues with deriving RH from q, I then tried using RHmax and RHmin from UI Metdata as inputs instead of deriving RH from q. All of these results are shown below. NFDRS/ represents my algorithm, UI Metdata DFM represents 100-hr DFM data from UI. 

![100-hr DFM](dfm_comparison_one_gridcell.png) 

Correlations are the following: 

r[UI DFM,NFDRS/UI Inputs] = 0.65

r[UI DFM,NFDRS/UI Inputs with RH] = 0.725

r[UI DFM,NFDRS/IS Inputs] = 0.39

r[NFDRS/UI Inputs,NFDRS/IS Inputs] = 0.61

** ISSUES TO FLAG ** 
1. Variability is very sensitive to WETRAT (precip rate in in/hr). This is dependent on the climate class. Although variability is smaller for the climate class used here (2), the correlations are higher. 
2. The correlation improves when UI Metdata Inputs with RHmin and RHmax are used. This highlights possible problems with my estimation of RH. Additionally, there are times when RH > 100 in my algorithm, which obviously doesn't make sense. 
3. EMC values for my algorithm do seem a bit high based on the literature, but this might not be unreasonable. 